# 🧠 TD : Web scraping avec requests et BeautifulSoup


🎯 Objectifs :
Savoir extraire des données à partir d’un site web

Maîtriser la navigation dans une page HTML

Comprendre l’importance des sélecteurs CSS, des classes, des attributs

Traiter des cas réels (pagination, liens relatifs, nettoyage de texte)

# 🧩 Partie 1 — Découverte et extraction de base


🔍 Exercice 1 — Récupérer une page HTML
URL à utiliser : https://books.toscrape.com/

Envoie une requête GET à cette URL.

Vérifie le code de réponse.

Affiche les 200 premiers caractères du HTML.

Bonus : Ajoute un header User-Agent pour simuler un navigateur.

🔍 Exercice 2 — Extraire les titres des livres

Analyse la page (ouvre la dans ton navigateur).

Utilise BeautifulSoup pour récupérer tous les titres des livres affichés.

Affiche les titres un par un dans la console.

Indice : Les livres sont dans des balises " article class="product_pod"."

## exercice 1

2 verifer le code de la reponse 

In [1]:
from bs4 import BeautifulSoup as bs 
import requests as r 

url ="https://books.toscrape.com/"

header={"User-Agent":"Mozilla/5.0"}
response=r.get(url,headers=header)

print(response.status_code)



200


3 afficher les 200 premier caractere du html 

In [2]:
#
html=response.text
print(html[:200])



<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if I


## exercice 2 

recuperation des titre des livres affichés sur le site web 

In [3]:
soup =bs(html,"html.parser")
books= soup.find_all('article',class_="product_pod")
titre=[]
for book in books:
    titre.append(book.h3.a["title"])
print(titre)

['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind', 'The Requiem Red', 'The Dirty Little Secrets of Getting Your Dream Job', 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'The Black Maria', 'Starving Hearts (Triangular Trade Trilogy, #1)', "Shakespeare's Sonnets", 'Set Me Free', "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Rip it Up and Start Again', 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991', 'Olio', 'Mesaerion: The Best Science Fiction Stories 1800-1849', 'Libertarianism for Beginners', "It's Only the Himalayas"]


# PARTI 2

##  Exercice 3 — Extraire le titre, le prix, la disponibilité

extraction des prix 

In [4]:
prix=soup.find_all('p',class_="price_color")
#print(prix)
price=[]

for montant in prix:
    price.append(montant.get_text())
print(price)



['Â£51.77', 'Â£53.74', 'Â£50.10', 'Â£47.82', 'Â£54.23', 'Â£22.65', 'Â£33.34', 'Â£17.93', 'Â£22.60', 'Â£52.15', 'Â£13.99', 'Â£20.66', 'Â£17.46', 'Â£52.29', 'Â£35.02', 'Â£57.25', 'Â£23.88', 'Â£37.59', 'Â£51.33', 'Â£45.17']


supprimons le caracte "Â" 

In [9]:
price= [p.replace("Â", "").strip() for p in price]
price

['£51.77',
 '£53.74',
 '£50.10',
 '£47.82',
 '£54.23',
 '£22.65',
 '£33.34',
 '£17.93',
 '£22.60',
 '£52.15',
 '£13.99',
 '£20.66',
 '£17.46',
 '£52.29',
 '£35.02',
 '£57.25',
 '£23.88',
 '£37.59',
 '£51.33',
 '£45.17']

extraction des disponibilités 

In [11]:
disponibilités =soup.find_all('p',class_="instock availability")
statut=[]

for element in disponibilités:
    statut.append(element.get_text().strip())
statut

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

nous allons concatener le tout dans un data fame pour avoir une sortie comme celle ci : 

Titre : A Light in the ...


Prix : £51.77


Disponibilité : In stock


In [14]:
import pandas as pd 

informations= {
    "titre":titre,
    "prix":price,
    "disponibilités": statut
}

test=pd.DataFrame(informations)
test.head()

,titre,prix,disponibilités
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock


# 🧩 Partie 3 — Pagination


### 🔁 Exercice 4 — Extraire les livres de plusieurs pages

Reprends l’exercice précédent.
Parcours les 10 premières pages du site.

Pour chaque livre, récupère les infos (titre, prix, disponibilité).

Stocke-les dans une liste de dictionnaires.

In [25]:
import time 

base_url = "https://books.toscrape.com/catalogue/page-{}.html"
all_books = []

for page in range(11):  
    url = base_url.format(page)
    response = r.get(url, headers=header)
    soup = bs(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        availability = book.find("p", class_="instock availability").text.strip()
        
        all_books.append({
            "Titre": title,
            "Prix": price,
            "Disponibilité": availability
        })
        
    # pour ne pas spammer le site
    time.sleep(1)  


In [26]:
all_books

[{'Titre': 'A Light in the Attic',
  'Prix': 'Â£51.77',
  'Disponibilité': 'In stock'},
 {'Titre': 'Tipping the Velvet',
  'Prix': 'Â£53.74',
  'Disponibilité': 'In stock'},
 {'Titre': 'Soumission', 'Prix': 'Â£50.10', 'Disponibilité': 'In stock'},
 {'Titre': 'Sharp Objects', 'Prix': 'Â£47.82', 'Disponibilité': 'In stock'},
 {'Titre': 'Sapiens: A Brief History of Humankind',
  'Prix': 'Â£54.23',
  'Disponibilité': 'In stock'},
 {'Titre': 'The Requiem Red', 'Prix': 'Â£22.65', 'Disponibilité': 'In stock'},
 {'Titre': 'The Dirty Little Secrets of Getting Your Dream Job',
  'Prix': 'Â£33.34',
  'Disponibilité': 'In stock'},
 {'Titre': 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  'Prix': 'Â£17.93',
  'Disponibilité': 'In stock'},
 {'Titre': 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  'Prix': 'Â£22.60',
  'Disponibilité': 'In stock'},
 {'Titre': 'The Black Maria', 'Prix': 'Â£52.15', 'Dispon


# 🧠 Exercice 5 — Explorer les liens des livres


Pour chaque livre, récupère le lien vers sa page de détail.

Visite cette page.

Récupère la description du livre (si présente).

📦 Bonus : idées d’extensions
Ajouter un délai entre les requêtes (avec time.sleep) pour ne pas surcharger le site.

Utiliser try/except pour éviter les erreurs si un élément est manquant.

Créer une petite interface (avec Streamlit par exemple) pour afficher les résultats.

In [27]:
import csv 

In [29]:
base_url = "https://books.toscrape.com/catalogue/"
page_url = "page-{}.html"
book_data = []


#2 page pour l'exemple 
for page in range(1, 3):  
    url = base_url + page_url.format(page)
    response = r.get(url, headers=header)
    soup = bs(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        availability = book.find("p", class_="instock availability").text.strip()
        
        # Lien vers la page de détail (relatif)
        relative_link = book.h3.a["href"]
        detail_url = base_url + relative_link.replace('../../../', '')
        
        # Visite de la page de détail
        detail_response = r.get(detail_url, headers=header)
        detail_soup = bs(detail_response.text, "html.parser")

        description_tag = detail_soup.find("meta", attrs={"name": "description"})
        description = description_tag["content"].strip() if description_tag else "Aucune description"

        book_data.append({
            "Titre": title,
            "Prix": price,
            "Disponibilité": availability,
            "Description": description
        })

        time.sleep(0.5)  # Respect du serveur



In [30]:
# Sauvegarde en CSV avec description
with open("books_detail.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["Titre", "Prix", "Disponibilité", "Description"])
    writer.writeheader()
    writer.writerows(book_data)


In [31]:
book_data

[{'Titre': 'A Light in the Attic',
  'Prix': 'Â£51.77',
  'Disponibilité': 'In stock',
  'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you 